# Función final para identificación del Liker

In [78]:
# Instalacionde requerimientos en caso de ser necesario
# pip install -r requirements.txt 

In [88]:
# Carga de librerias necesarias
# Importacion de librerias
import os
import spacy
import pandas as pd
import numpy as np
from spacy.matcher import Matcher

# Carga del modelo
spacy.require_cpu() # Temporal mientras se arregla problema con uso de GPU
nlp = spacy.load("es_core_news_lg")

In [89]:
# La función ya trae por defecto un conjunto de patrones para identificar el Liker pero solo es válido para la campaña de SURA
def Speaker_Asignation(file_path, additional_patterns = [], write_txt = True, keep_strategies = True, Word_Count = False, Num_Matches = False, install_requirements = False):
    """
    file_path: path of the transcripted .txt file without the Speaker asignation (i.e each file has either SPEAKER_00 or SPEAKER_01)
    write_txt: Specify if you want to create a new .txt file with the asigned roles by the function
    additional_patterns: List or dict. Additional patterns to be used in the third strategy (those one not considered or belong to other campaign).
    keep_strategies: Include columns of the asigned role in each strategy
    Word_Count: Include the number of words spoken by speaker in each line
    Num_Matches: Include the number of matches in the considered patterns to search
    """
    # Lectura del archivo de transcripcion
    with open(file_path, "r", encoding = "utf-8") as archivo:
        aux_list_df = [] # Lista auxiliar para creacion del df
        # Itera sobre cada línea del archivo
        for linea in archivo:
            # Creacion del dataframe con la transcripcion
            transcript = [linea[:17].strip(), linea[19:29].strip(), linea[31:].strip()]
            aux_list_df.append(transcript)
            
        # Creacion del dataframe con la transcripcion
        transcript_df = pd.DataFrame(aux_list_df, columns = ["Tiempo", "Speaker", "Texto"])
    
    # Solucion 1: Liker = Primer speaker
    transcript_df["Speaker_Asignado_Opt_1"] = np.where(transcript_df.Speaker == transcript_df.Speaker[0], "Liker", "Cliente")

    # Solucion 2: Liker =  Persona que mas habla
    transcript_df["Word_Count"] = transcript_df.Texto.apply(lambda x: len(x.split())) # Columna auxiliar para contar el numero de palabras en cada fila

    # Group by para identificar el Speaker que más habla
    words_per_speaker = transcript_df[["Speaker", "Word_Count"]].groupby(by = "Speaker").sum().reset_index()

    # Asignacion del speaker segun que tanto habla
    # Lista booleana con el respectivo mapeo
    mapping_opt_2 = [words_per_speaker[words_per_speaker.Word_Count == words_per_speaker.Word_Count.max()].Speaker == transcript_df.Speaker[i] for i in range(len(transcript_df.Speaker))]
    transcript_df["Speaker_Asignado_Opt_2"] = np.where(mapping_opt_2, "Liker", "Cliente")

    # Eliminacion de columna auxiliar (Eliminar linea de abajo en caso de querer conservarla)
    if not Word_Count:
        transcript_df.drop(columns = "Word_Count", inplace = True)

    # Solucion 3: Conteo del numero de ocurrencias de patrones particulares
    # Creacion del matcher para buscar patrones
    matcher = Matcher(nlp.vocab) # Matcher
    sura_patterns = [[{"LOWER": "sudamericana"}], [{"LOWER": "de"}, {"LOWER": "sudamericana"}],
                    [{"LOWER": {"REGEX": r'sura'}}], [{"LOWER": "de"}, {"LOWER": {"REGEX": r'sura'}}],
                    [{"LOWER": {"REGEX": r'seguro'}}], [{"LOWER": {"REGEX": r'seguro'}}, {"LOWER": {"REGEX": r'sura'}}],
                    [{"LOWER": "póliza"}], [{"LOWER": "poliza"}], [{"LOWER": {"REGEX": "cotización"}}],
                    [{"LOWER": {"REGEX": r'asegura'}}],
                    [{"LOWER": "grabada"}], [{"LOWER": "monitoreada"}], [{"LOWER": "siendo"}, {"LOWER": "grabada"}],
                    [{"LOWER": "siendo"}, {"LOWER": "grabada"}, {"LOWER": "y"}, {"LOWER": "monitoreada"}],
                    [{"LOWER": "validar"}, {"LOWER": "datos"}]]
    matcher.add("sura_patterns", sura_patterns)
    
    # Agregacion de patrone adicionales dados por el usuario
    if len(additional_patterns) != 0:
        if isinstance(additional_patterns, list):
            additional_patterns_lower = [token.lower() for token in additional_patterns]
            patterns = [[{"LOWER": token} for token in item.split()] for item in additional_patterns_lower]
            print(patterns)
            matcher.add("additional_patterns", patterns)
        elif isinstance(additional_patterns, dict):
             matcher.add(additional_patterns)

    # Conteo de cuantos patrones encuentra por speaker
    transcript_df["Num_Matches"] = transcript_df.Texto.apply(lambda x: len(matcher(nlp(x))))

    # Group by para identificar el Speaker que mas patrones repite
    num_matches = transcript_df[["Speaker", "Num_Matches"]].groupby(by = "Speaker").sum().reset_index()

    # Asignacion del speaker segun numero de matches
    # Lista booleana con el respectivo mapeo
    mapping_opt_3 = [num_matches[num_matches.Num_Matches == num_matches.Num_Matches.max()].Speaker == transcript_df.Speaker[i] for i in range(len(transcript_df.Speaker))]
    transcript_df["Speaker_Asignado_Opt_3"] = np.where(mapping_opt_3, "Liker", "Cliente")

    # Eliminacion de columna auxiliar (Eliminar linea de abajo en caso de querer conservarla)
    if not Num_Matches:
        transcript_df.drop(columns = "Num_Matches", inplace = True)

    # Asignacion final del speaker
    bool_df = transcript_df[["Speaker_Asignado_Opt_1", "Speaker_Asignado_Opt_2", "Speaker_Asignado_Opt_3"]].apply(lambda x: x == "Liker")
    transcript_df["Speaker_Asignado"] = np.where(np.sum(bool_df, axis = 1).between(2, 3), "[Liker  ]:", "[Cliente]:")
    
    # Elimnacion de columnas para asignacion final (en caso de requerirse)
    if not keep_strategies:
        transcript_df.drop(columns = ["Speaker_Asignado_Opt_1", "Speaker_Asignado_Opt_2", "Speaker_Asignado_Opt_3"], inplace = True)

    # Escribir la transcripción en un archivo txt
    if write_txt:
        # Lista para   
        to_write_list = transcript_df[["Tiempo", "Speaker_Asignado", "Texto"]].apply(lambda x: " ".join(x.astype(str)), axis = 1)
        # Abrir el archivo en modo de escritura
        with open("Identified_Speakers\\" + "Asigned_Speaker_" + os.path.basename(file_path), 'w') as archivo:
            # Iterar sobre la lista y escribir cada texto en una nueva línea
            for texto in to_write_list:
                archivo.write(texto + '\n')

    # print("Los textos se han escrito en el archivo {}.".format(nombre_archivo))
    return {"Transcripted_Df": transcript_df, "Word_Count": words_per_speaker, "Num_Matches": num_matches}

# Asignación de roles

In [91]:
Speaker_Asignation("Raw_Transcriptions/1001764369_1_transcription.txt", write_txt=False, keep_strategies=False)["Transcripted_Df"]

,Tiempo,Speaker,Texto,Speaker_Asignado
0,0:00:10 - 0:00:10,SPEAKER_01,Hola,[Cliente]:
1,0:00:11 - 0:00:13,SPEAKER_00,"Sí, buenas tardes ¿Con Juan Cartagena?",[Liker ]:
2,0:00:14 - 0:00:15,SPEAKER_01,Sí,[Cliente]:
3,0:00:16 - 0:00:18,SPEAKER_00,"Hola, hola ¿Cómo estás?",[Liker ]:
4,0:00:19 - 0:00:20,SPEAKER_01,Muy bien,[Cliente]:
...,...,...,...,...
80,0:09:43 - 0:09:44,SPEAKER_01,Listo. Listo.,[Cliente]:
81,0:09:44 - 0:09:50,SPEAKER_00,"Listo. Y dime, yo ya me comunicaría entonces c...",[Liker ]:
82,0:09:51 - 0:09:53,SPEAKER_01,Bueno. Listo. Hágale pues.,[Cliente]:
83,0:09:53 - 0:09:56,SPEAKER_00,"Bueno, Juan. Ustedes recuerden que hablaste co...",[Liker ]:
